<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Follow company followers monthly
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Follow_connections_from_profile.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #network #connections #naas_drivers #analytics #csv #html #image #content #plotly

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

## Input

### Get common variables, functions

In [ ]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

### Setup Variables

In [ ]:
KPI_TITLE = "Followers"

### Setup Constantes

In [ ]:
# Inputs
PLATFORM = LINKEDIN
OWNER = LK_COMPANY_NAME
LOGO = LINKEDIN_LOGO
COLOR = LINKEDIN_COLOR
COLOR2 = LINKEDIN_COLOR2
PERIOD = PERIOD_MTD
PERIOD_D = DISPLAY_MTD
PERIOD_TEXT = TEXT_MTD
PERIOD_TEXT2 = TEXT2_MTD
SCENARIO = SCENARIO_MTD
TITLE = f"{KPI_TITLE.lower().capitalize()} over the past {str(MONTH_ROLLING)} {PERIOD_TEXT2}"
COL_DATE = "FOLLOWED_AT"

# Outputs
name_output = f"LINKEDIN_FOLLOW_COMPANY_FOLLOWERS_{SCENARIO}_{LK_COMPANY_ID}"
csv_output = path.join(LINKEDIN_OUTPUTS, f"{name_output}.csv")
html_output = path.join(LINKEDIN_OUTPUTS, f"{name_output}.html")
image_output = path.join(LINKEDIN_OUTPUTS, f"{name_output}.png")

## Model

### Get followers from company

In [ ]:
df_init = get_data(LK_COMPANY_FOLLOWERS)
print("Data fetched:", len(df_init))
df_init.head(1)

### Prep trend data

In [ ]:
def get_trend(df_init,
              col_date,
              col_value,
              agg_value):
    # Init variable
    df = df_init.copy()
    
    # Groupby period
    df[col_date] = pd.to_datetime(df[col_date]).dt.strftime(PERIOD)
    df = df.groupby(col_date, as_index=False).agg({col_value: agg_value})
    
    # Rename column
    to_rename = {
        col_date: "DATE",
        col_value: "VARV"
    }
    df = df.rename(columns=to_rename)
    df.insert(loc=1, column="VALUE", value=df["VARV"].cumsum())
    df.insert(loc=2, column="VALUE_COMP", value=df["VALUE"] - df["VARV"])
    df["VARP"] = df["VARV"] / df["VALUE_COMP"]
    
    # Add last month in not in dataframe
    if df.loc[df.index[-1], "DATE"] != datetime.now().strftime(PERIOD):
        tmp_df = df[-1:].reset_index(drop=True)
        tmp_df.loc[0, "DATE"] = datetime.now().strftime(PERIOD)
        tmp_df.loc[0, "VALUE"] = 0
        df = pd.concat([df, tmp_df]).reset_index(drop=True)
    
    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE"], format=PERIOD).dt.strftime(PERIOD_D)
    
    # Plotly: Value display
    df["VALUE_D"] = "<b><span style=font-size: 16px; style=font-family: Arial>" + df["VALUE"].map("{:,.0f}".format).str.replace(",", " ") + "</span></b>"
    
    # Plotly: Variation display
    df["VARV_D"] = df["VARV"].map("{:,.0f}".format).str.replace(",", " ")
    df.loc[df["VARV"] >= 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = df["VARP"].map("{:,.0%}".format).str.replace(",", " ")
    df.loc[df["VARP"] >= 0, "VARP_D"] = "+" + df["VARP_D"]
    
    # Plotly: hovertext
    df["TEXT"] = ("<b><span style='font-size: 14px;'>" + df["DATE_D"].astype(str) + ": " + df["VALUE_D"] + "</span></b><br>"
                  "<span style='font-size: 12px;'>" + f"{PERIOD_TEXT}: " + df["VARV_D"] + " (" + df["VARP_D"] + ")</span>")
    
    # Return month rolling
    return df[-MONTH_ROLLING:].reset_index(drop=True)

df_actual = get_trend_barline(df_init,
                              col_date=COL_DATE,
                              col_value="PROFILE_ID",
                              agg_value="count")
df_actual

### Create barlinechart

In [ ]:
fig = create_barlinechart(df_actual,
                          yaxes_left="New followers",
                          yaxes_right="Total followers")

## Output

### Save and share your csv file

In [ ]:
# Save your dataframe in CSV
output_path = save_graph_asset(df_actual, csv_output)

# Share output with naas
csv_link = naas.asset.add(output_path)

#-> Uncomment the line below to remove your asset
# naas.asset.delete(csv_output)

### Save and share your graph in HTML

In [ ]:
# Save your graph in HTML
output_path = save_graph_asset(fig, html_output)

# Share output with naas
html_link = naas.asset.add(output_path, params={"inline": True})

#-> Uncomment the line below to remove your asset
# naas.asset.delete(html_output)

### Save and share your graph in image

In [ ]:
# Save your graph in PNG
output_path = save_graph_asset(fig, image_output)

# Share output with naas
image_link = naas.asset.add(output_path)

#-> Uncomment the line below to remove your asset
# naas.asset.delete(image_output)

### Update Notion Report

In [ ]:
update_report_status(KPI_TITLE,
                     PLATFORM,
                     OWNER,
                     df_actual,
                     image_link=image_link,
                     html_link=html_link,
                     csv_link=csv_link)